In [6]:
import json
import random
from IPython.display import clear_output
from threading import Thread, Lock
from datetime import datetime
import time
import numpy as np
from shapely.geometry import Polygon, Point

mutex=Lock()

In [7]:
with open('./json_info_sezioni.json') as json_file:
    info_sezioni = json.load(json_file)

In [8]:
with open('./json_agenti_res.json') as json_file:
    agenti_attivi = json.load(json_file)

In [9]:
#Individuo numero di abitanti per sezione e verifico che nessuno ecceda della stima calcolata
sezioni=[0]*6108

for a in agenti_attivi:
    sez_residenza=a['sez_residenza']
    sezioni[sez_residenza]=sezioni[sez_residenza]+1

for s in info_sezioni:
    if s['abitanti']<sezioni[s['sezione']]:
        print("sez. num: "+str(s['sezione']))
    

In [13]:
#Calcolo numero di non attivi generabili (10-15% per ogni sezione dai possibili)
non_attivi=[0]*6108

for s in info_sezioni:
    attivi_stima=s['num_eta_attiva_15-64']
    over65_stima=s['num_anziani_over65']
    non_attivi[s['sezione']]=int(round((attivi_stima-sezioni[s['sezione']]+over65_stima)*random.randint(15,20)/100,0))
    
    

In [14]:
#Numero agenti non attivi
sum(non_attivi)

165268

In [15]:
#Trovo ultimo id agenti
ids=[]
for a in agenti_attivi:
    ids.append(a['id'])
max(ids)

210736

In [16]:
id_a=210737 #Ultimo id agenti_attivi
agenti_non_attivi=[]

#Creo agenti non attivi
for s in info_sezioni:
    num_non_attivi=non_attivi[s['sezione']]
    for i in range(0,num_non_attivi):
        a={}
        a['id']=id_a
        id_a=id_a+1
        a['tipo']='non_attiva'
        a['categoria']=random.choice(['lavoratore','over65'])
        a['sez_residenza']=s['sezione']
        a['automunito']=random.choice([True,False])
        agenti_non_attivi.append(a)


In [17]:
#Salvo agenti non attivi
with open('./json_agenti_non_attivi.json', 'w') as fp:
        json.dump(agenti_non_attivi, fp)

_____________

In [18]:
#Trovo ultimo id famiglia
ids=[]
for a in agenti_attivi:
    ids.append(a['id_fam'])
max(ids)

105258

In [19]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
mutex=Lock()

#Assegno o creo nuclei famigliari
id_fam=105259

def crea_agente(a):
#for a in agenti_non_attivi:
    global mutex
    global id_fam
    
    sez_residenza=a['sez_residenza']
    nuclei_stessa_sez=[]
    
    attivi_stessa_sez=list(filter(lambda at: at['sez_residenza'] == sez_residenza, agenti_attivi))
    for at in attivi_stessa_sez:
        nuclei_stessa_sez.append(at['id_fam'])
        
    non_attivi_stessa_sez=list(filter(lambda nat: nat['sez_residenza'] == sez_residenza, agenti_non_attivi))
    for nat in non_attivi_stessa_sez:
        if 'id_fam' in nat:
            nuclei_stessa_sez.append(nat['id_fam'])
        
    if random.choice([True,True,False]) and len(nuclei_stessa_sez)!=0: #Assegno agente ad un nucleo già esistente 
        a['id_fam']=random.choice(nuclei_stessa_sez)
    else:
        mutex.acquire()
        a['id_fam']=id_fam
        id_fam=id_fam+1
        mutex.release()

#Inizializzazione Threads
print("Inizializzazione lista threads")
threads = []

for a in agenti_non_attivi:
    t=Thread(target=crea_agente, args=(a,))
    threads.append(t)

#Avvio i threads
print("Avvio threads")
for x in threads:
    x.start()

#Attendo che tutti abbiano terminato
print("Attendo che tutti abbiano terminato")
for x in threads:
    x.join()
        
print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-24 09:56:56
Inizializzazione lista threads
Avvio threads
Attendo che tutti abbiano terminato
Fine = 2021-03-24 11:45:11


In [20]:
#Salvo agenti non attivi con famiglia
with open('./json_agenti_non_attivi_fam.json', 'w') as fp:
        json.dump(agenti_non_attivi, fp)

In [21]:
#Controllo elaborazione
id_fam_c=0
count=0
tipo=[]
for a in agenti_non_attivi:
    id_fam=a['id_fam']
    if id_fam_c==id_fam:
        count=count+1
    else:
        tipo.append(count)
        id_fam_c=id_fam
        count=1

for a in agenti_attivi:
    id_fam=a['id_fam']
    if id_fam_c==id_fam:
        count=count+1
    else:
        tipo.append(count)
        id_fam_c=id_fam
        count=1
        
from collections import Counter

Counter(tipo)
    

Counter({0: 1,
         1: 187355,
         2: 47179,
         3: 29224,
         4: 59,
         5: 21,
         6: 13,
         8: 1,
         10: 1,
         7: 2,
         11: 1})

In [22]:
#importo json agenti non attivi
with open('./json_agenti_non_attivi_fam.json') as json_file:
    agenti_non_attivi = json.load(json_file)

In [23]:
#importo json agenti attivi
with open('./json_agenti_res.json') as json_file:
    agenti_attivi = json.load(json_file)

In [24]:
with open('/Volumes/HDD/Università/Tirocinio/Dati censo/sezioni_censimento_2011.geojson') as file:
    gj_sezioni = json.load(file)
features_sezioni=gj_sezioni['features']

In [25]:
#Funzione per generare punto casuale in un poligono
def random_point_within(poly):
    min_x, min_y, max_x, max_y = poly.bounds
    
    while True:
        random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            return random_point

In [28]:
#Trovo ultimo id famiglia
ids=[]
for a in agenti_non_attivi:
    ids.append(a['id_fam'])
max(ids)-105258

55006

In [30]:
#Assegno residenza alle famiglie
famiglie=[None] * (55007)
for an in agenti_non_attivi:
    if an['id_fam']>=105259:
        sezione=list(filter(lambda s: s['properties']['SEZ'] == an['sez_residenza'],features_sezioni))
        poly = Polygon(sezione[0]['geometry']['coordinates'][0])
        point = random_point_within(poly)
        famiglie[an['id_fam']-105258]=point

In [32]:
#Assegna residenza
for na in agenti_non_attivi:
    id_fam=na['id_fam']
    if id_fam >= 105259: #nuove residenze
        na['X_residenza']=famiglie[id_fam-105258].x
        na['Y_residenza']=famiglie[id_fam-105258].y
    else: #fam già creata
        fam=list(filter(lambda a: a['id_fam'] == id_fam,agenti_attivi))
        na['X_residenza']=fam[0]['X_residenza']
        na['Y_residenza']=fam[0]['Y_residenza']


In [33]:
#Salvo agenti non attivi con residenza
with open('./json_agenti_non_attivi_res.json', 'w') as fp:
        json.dump(agenti_non_attivi, fp)